In [1]:
import os
import sys
import json
import pandas as pd

In [2]:
import copy
import os
import re
from typing import Union, Tuple

import numpy as np

In [3]:
def add_path(path):
    if path not in sys.path:
        sys.path.append(path)

In [4]:
root = os.getcwd().split("NatGen")[0]
root

'/home/mdrahman/projects/'

In [5]:
add_path(f"{root}NatGen")

In [6]:
from src.data_preprocessors import *

from src.data_preprocessors.language_processors import JavaAndCPPProcessor
from src.data_preprocessors.transformations.transformation_base import TransformationBase

language = 'c'
parser_path = root + '/NatGen/parser/languages.so'


In [164]:
import re
import copy

import numpy as np
from tree_sitter import Node

from src.data_preprocessors.language_processors.utils import get_tokens, dfs_print


class JavaAndCPPProcessor:

    @classmethod
    def extract_if_else(cls, root, code_str, operator_list):
        ext_opt_list = ["&&", "&", "||", "|"]
        expressions = []
        queue = [root]

        not_consider = []
        while len(queue) > 0:
            current_node = queue[0]
            queue = queue[1:]
            if str(current_node.type) == 'if_statement':
                clause = code_str[current_node.start_byte:current_node.end_byte].decode()
                des = (current_node.children)[1]
                cond = code_str[des.start_byte:des.end_byte].decode()
                stack = [des]
                nodes = []
                while len(stack) > 0:
                    root1 = stack.pop()
                    if len(root1.children) == 0:
                        nodes.append(root1)

                    for child in root1.children:
                        stack.append(child)
                nodes.reverse()
                counter = 0
                extra_counter = 0
                for w in nodes:
                    if str(w.type) in operator_list:
                        counter = counter + 1
                    if str(w.type) in ext_opt_list:
                        extra_counter = extra_counter + 1
                if not (counter == 1 and extra_counter == 0):
                    continue
                children_nodes = current_node.children
                flagx = 0
                flagy = 0
                for w in children_nodes:
                    if str(w.type) == "else":
                        flagx = 1
                    if str(w.type) == "if_statement":
                        not_consider.append(w)
                        flagy = 1
                if flagx == 1 and flagy == 0:
                    expressions.append([current_node, des])
            for child in current_node.children:
                if child not in not_consider:
                    queue.append(child)

        return expressions
    
    @classmethod
    def get_tokens_for_multi_blockswap(cls, code, root, block_change, parent=None):
        if isinstance(code, str):
            code = code.encode()
        assert isinstance(root, Node)
        tokens = []
        
        if root.type == "comment":
            return tokens, None
        if "string" in str(root.type):
            return [code[root.start_byte:root.end_byte].decode()], None
        children = root.children
        if len(children) == 0:
            if root in block_change[2]:
                op = code[root.start_byte:root.end_byte].decode()
                if op == "<":
                    tokens.append(">=")
                elif op == ">":
                    tokens.append('<=')
                elif op == ">=":
                    tokens.append('<')
                elif op == "<=":
                    tokens.append('>')
                elif op == "!=":
                    tokens.append('==')
                elif op == "==":
                    tokens.append('!=')
            else:
                tokens.append(code[root.start_byte:root.end_byte].decode())
        if parent is not None and parent.type == 'switch_statement' and root.type=='compound_statement':
            case_positions = []
            for i, child in enumerate(children):
                if child.type == 'case_statement' and ('break' in code[child.start_byte:child.end_byte].decode()):
                    case_positions.append(i)
            case_positions = case_positions[:-1]
            copied = copy.deepcopy(case_positions)
            np.random.shuffle(copied)
            # print(code[case_positions[-1].start_byte: case_positions[-1].end_byte])
            # print(children)
            # print(root.type, code[root.start_byte:root.end_byte].decode())
            for i, child in enumerate(children):
                child_type = str(child.type)
                # print(child, child_type)
                if child_type == 'case_statement' and (i in copied):
                    idx = copied.index(i)
                    ts, _ = cls.get_tokens_for_multi_blockswap(code, children[case_positions[idx]], block_change, parent=root)
                else:
                    ts, _ = cls.get_tokens_for_multi_blockswap(code, child, block_change, parent=root)
                tokens += ts
        elif root.type == 'binary_expression' and len(children) == 3 and children[1].type in ('+', '*'):
            for child in children[::-1]:
                ts, _ = cls.get_tokens_for_multi_blockswap(code, child, block_change, parent=root)
                tokens += ts
        else:
            for child in children:
                child_type = str(child.type)
                # print(child, child_type)
                if child in block_change[0]:
                    idx = block_change[0].index(child)
                    ts, _ = cls.get_tokens_for_multi_blockswap(code, block_change[1][idx], block_change, parent=root)
                else:
                    ts, _ = cls.get_tokens_for_multi_blockswap(code, child, block_change, parent=root)
                tokens += ts
        # if root.type == 'declaration':
        #     text = code[root.start_byte:root.end_byte].decode()
        #     if '=' not in text:
        # print(parent.type)
        # if parent is not None and parent.type == 'switch_statement' and root.type=='compound_statement':
        #     print(children)
        #     print(root.type, code[root.start_byte:root.end_byte].decode())
        return tokens, None


    @classmethod
    def multi_block_swap_c(cls, code_str, parser):
        code = code_str.encode()
        root = parser.parse_code(code)
        operator_list = ['<', '>', '<=', '>=', '==', '!=']
        pair = cls.extract_if_else(root, code, operator_list)
        success = False
        lst = list(range(0, len(pair)))
        try:
            block_change = [[], [], []]
            for selected in lst:
                clause = pair[selected][0]
                des = pair[selected][1]
                st = [des]
                nodes = []
                while len(st) > 0:
                    root1 = st.pop()
                    if len(root1.children) == 0:
                        nodes.append(root1)
                        if (code[root1.start_byte:root1.end_byte].decode()) in operator_list:
                            opt_node = root1
                            break
                    for child in root1.children:
                        st.append(child)
                nodes = clause.children
                flag = 0
                first_block, second_block = None, None
                for current_node in nodes:
                    if str(current_node.type) == 'compound_statement':
                        if flag == 0:
                            first_block = current_node
                            flag = 1
                        else:
                            second_block = current_node
                if first_block and second_block:
                    block_change[0].append(first_block)
                    block_change[1].append(second_block)
                    block_change[0].append(second_block)
                    block_change[1].append(first_block)
                    block_change[2].append(opt_node)
            try:
                code_list = \
                    cls.get_tokens_for_multi_blockswap(code, root, block_change)[0]
                code_string = ""
                for w in code_list:
                    code_string = code_string + w + " "
                code_string = code_string.strip()
                success = True
            except Exception as E:
                print("Exception: ", str(E))
                success = False
        except Exception as E:
            print("Exception occurred", str(E))
            pass
        if not success:
            code_string = cls.beautify_java_code(get_tokens(code_str, root))
        return code_string, success


In [165]:


processor_function = {"c": [JavaAndCPPProcessor.multi_block_swap_c],}


class BlockSwap(TransformationBase):
    """
    Swapping if_else block
    """

    def __init__(self, parser_path, language):
        super(BlockSwap, self).__init__(parser_path=parser_path, language=language)
        self.language = language
        self.transformations = processor_function[language]
        processor_map = {
            "c": self.get_tokens_with_node_type,
        }
        self.final_processor = processor_map[self.language]

    def transform_code(
            self,
            code: Union[str, bytes],
    ) -> Tuple[str, object]:
        success = False
        transform_functions = copy.deepcopy(self.transformations)
        while not success and len(transform_functions) > 0:
            function = np.random.choice(transform_functions)
            transform_functions.remove(function)
            modified_code, success = function(code, self)
            if success:
                code = modified_code
        root_node = self.parse_code(
            code=code
        )
        return_values = self.final_processor(
            code=code.encode(),
            root=root_node
        )
        if isinstance(return_values, tuple):
            tokens, types = return_values
        else:
            tokens, types = return_values, None
        # print(tokens)
        return  " ".join(tokens), \
               {
                   "types": types,
                   "success": success
               }


In [184]:
c_code = """
#include<stdio.h>
#define y  9
int process(int option, int a, int b) {
    int *x;
    int k = a + b;
    int result = k;

    if (b != 0) {
        result = a / b;
    } else {
        printf("Error: Division by zero is undefined.\n");
        return -1;  // Return an error code
    }

    switch (option) {
        case 1:
            result = a + b * c;
            break;
        case 2:
            result = ((a + b) - c);
            break;
        case 3:
            result = a * c;
            break;
        default:
            printf("Error: Invalid option. Please choose a valid option (1-4).\n");
            return -1;  // Return an error code
    }
    
    return result;
}
"""

In [185]:
' '.join(c_code.split())

'#include<stdio.h> #define y 9 int process(int option, int a, int b) { int *x; int k = a + b; int result = k; if (b != 0) { result = a / b; } else { printf("Error: Division by zero is undefined. "); return -1; // Return an error code } switch (option) { case 1: result = a + b * c; break; case 2: result = ((a + b) - c); break; case 3: result = a * c; break; default: printf("Error: Invalid option. Please choose a valid option (1-4). "); return -1; // Return an error code } return result; }'

In [175]:
def beautify(code):
    space = ''
    final_code = ''
    flag = False
    string_on = False
    case_on = False
    for ch in code:
        # print(ch)
        if string_on:
            final_code += ch
            if ch == "\"":
                string_on = False
            continue
        elif ch == "\"":
            final_code += "\""
            string_on = True
            continue
        if ch == '{':
            space += '    '
            final_code += "{\n"
            flag = True
        elif ch == '}':
            space = space[:-3]
            final_code += "\n" + space + ch + "\n"
            flag = True
        elif ch == ';':
            if len(final_code) > 0 and final_code[-1] == '\n':
                final_code = final_code[:-1]
            final_code += ch + "\n"
            flag = True
        elif ch == ':':
            final_code += ch + "\n"
            flag = True
        else:
            if flag:
                final_code += space[:-1]
                flag = False
            final_code += ch
    return final_code



In [176]:
transformer = BlockSwap(parser_path, language)

In [177]:
code, _ = transformer.transform_code(c_code)

['#include', '<stdio.h>', '\n', '#define', 'y', '  9', '\n', 'int', 'process', '(', 'int', 'option', ',', 'int', 'a', ',', 'int', 'b', ')', '{', 'int', '*', 'x', ';', 'int', 'k', '=', 'b', '+', 'a', ';', 'int', 'result', '=', 'k', ';', 'if', '(', 'b', '==', '0', ')', '{', 'printf', '(', '"Error: Division by zero is undefined.\n"', ')', ';', 'return', '-1', ';', '}', 'else', '{', 'result', '=', 'a', '/', 'b', ';', '}', 'switch', '(', 'option', ')', '{', 'case', '2', ':', 'result', '=', '(', '(', 'b', '+', 'a', ')', '-', 'c', ')', ';', 'break', ';', 'case', '1', ':', 'result', '=', 'c', '*', 'b', '+', 'a', ';', 'break', ';', 'case', '3', ':', 'result', '=', 'c', '*', 'a', ';', 'break', ';', 'default', ':', 'printf', '(', '"Error: Invalid option. Please choose a valid option (1-4).\n"', ')', ';', 'return', '-1', ';', '}', 'return', 'result', ';', '}']
Before strip,  #include <stdio.h> 
 #define y   9 
 int process ( int option , int a , int b ) { int * x ; int k = b + a ; int result = k ;

In [178]:
code

'#include <stdio.h> \n #define y    9  \n int process ( int option , int a , int b ) { int * x ; int k = b + a ; int result = k ; if ( b == 0 ) { printf ( "Error: Division by zero is undefined.\n" ) ; return -1 ; } else { result = a / b ; } switch ( option ) { case 2 : result = ( ( b + a ) - c ) ; break ; case 1 : result = c * b + a ; break ; case 3 : result = c * a ; break ; default : printf ( "Error: Invalid option. Please choose a valid option (1-4).\n" ) ; return -1 ; } return result ; }'

In [179]:
c_code

'\n#include<stdio.h>\n#define y  9\nint process(int option, int a, int b) {\n    int *x;\n    int k = a + b;\n    int result = k;\n\n    if (b != 0) {\n        result = a / b;\n    } else {\n        printf("Error: Division by zero is undefined.\n");\n        return -1;  // Return an error code\n    }\n\n    switch (option) {\n        case 1:\n            result = a + b * c;\n            break;\n        case 2:\n            result = ((a + b) - c);\n            break;\n        case 3:\n            result = a * c;\n            break;\n        default:\n            printf("Error: Invalid option. Please choose a valid option (1-4).\n");\n            return -1;  // Return an error code\n    }\n    \n    return result;\n}\n'

In [180]:
final_code = beautify(code)

In [181]:
print(final_code)

#include <stdio.h> 
 #define y    9  
 int process ( int option , int a , int b ) {
    int * x ;
    int k = b + a ;
    int result = k ;
    if ( b == 0 ) {
        printf ( "Error: Division by zero is undefined.
" ) ;
        return -1 ;
        
     }
     else {
         result = a / b ;
         
      }
      switch ( option ) {
          case 2 :
          result = ( ( b + a ) - c ) ;
          break ;
          case 1 :
          result = c * b + a ;
          break ;
          case 3 :
          result = c * a ;
          break ;
          default :
          printf ( "Error: Invalid option. Please choose a valid option (1-4).
" ) ;
          return -1 ;
          
       }
       return result ;
       
    }



In [108]:
final_code = code

In [109]:
print(final_code)

#include <stdio.h> 
 #define y    9  
 int process ( int option , int a , int b ) { int * x ; int k = a + b ; int result = k ; if ( b == 0 ) { printf ( "Error: Division by zero is undefined.
" ) ; return -1 ; } else { result = a / b ; } switch ( option ) { case 2 : result = a - c ; break ; case 1 : result = a + b * c ; break ; case 3 : result = a * c ; break ; default : printf ( "Error: Invalid option. Please choose a valid option (1-4).
" ) ; return -1 ; } return result ; }


In [110]:
print(c_code)


#include<stdio.h>
#define y  9
int process(int option, int a, int b) {
    int *x;
    int k = a + b;
    int result = k;

    if (b != 0) {
        result = a / b;
    } else {
        printf("Error: Division by zero is undefined.
");
        return -1;  // Return an error code
    }

    switch (option) {
        case 1:
            result = a + b * c;
            break;
        case 2:
            result = a - c;
            break;
        case 3:
            result = a * c;
            break;
        default:
            printf("Error: Invalid option. Please choose a valid option (1-4).
");
            return -1;  // Return an error code
    }
    
    return result;
}



In [118]:
from tree_sitter import Language, Parser

In [119]:
lang_object = Language(parser_path, language)
parser = Parser()
parser.set_language(lang_object)

In [193]:
def walk(root, code):
    if isinstance(code, str):
        code = code.encode()
    if "string" in str(root.type):
        return [code[root.start_byte:root.end_byte].decode()]
    children = root.children

    if len(children) == 0:
        return [code[root.start_byte:root.end_byte].decode()]
    if "if_statement" in root.type:
        print(root.type, code[root.start_byte:root.end_byte].decode())
    # print(children)
    ts = []
    for child in children:
        print(child)
        # if "binary_expression" in root.type and len(children) == 3:
        #     print("child", code[child.start_byte: child.end_byte].decode())
        ts += walk(child, code)
    return ts

In [194]:
c_code = """
#include<stdio.h>
#define y  9
int process(int option, int a, int b) {
    int *x;
    int k = a + b;
    int result = k;

    if (b > 0) result = a / b;
    else {
        printf("Error: Division by zero is undefined.\n");
        return -1;  // Return an error code
    }

    switch (option) {
        case 1:
            result = a + b + c;
            break;
        case 2:
            result = a - c;
            break;
        case 3:
            result = a * c;
            break;
        default:
            printf("Error: Invalid option. Please choose a valid option (1-4).\n");
            return -1;  // Return an error code
    }
    
    return result;
}
"""

In [195]:
tree =parser.parse(c_code.encode())

In [192]:
tokens = walk(tree.root_node, c_code)

<Node type=preproc_include, start_point=(1, 0), end_point=(2, 0)>
<Node type="#include", start_point=(1, 0), end_point=(1, 8)>
<Node type=system_lib_string, start_point=(1, 8), end_point=(1, 17)>
<Node type="
", start_point=(1, 17), end_point=(2, 0)>
<Node type=preproc_def, start_point=(2, 0), end_point=(3, 0)>
<Node type="#define", start_point=(2, 0), end_point=(2, 7)>
<Node type=identifier, start_point=(2, 8), end_point=(2, 9)>
<Node type=preproc_arg, start_point=(2, 9), end_point=(2, 12)>
<Node type="
", start_point=(2, 12), end_point=(3, 0)>
<Node type=function_definition, start_point=(3, 0), end_point=(32, 1)>
<Node type=primitive_type, start_point=(3, 0), end_point=(3, 3)>
<Node type=function_declarator, start_point=(3, 4), end_point=(3, 37)>
<Node type=identifier, start_point=(3, 4), end_point=(3, 11)>
<Node type=parameter_list, start_point=(3, 11), end_point=(3, 37)>
<Node type="(", start_point=(3, 11), end_point=(3, 12)>
<Node type=parameter_declaration, start_point=(3, 12), e

In [149]:
tokens

['#include',
 '<stdio.h>',
 '\n',
 '#define',
 'y',
 '  9',
 '\n',
 'int',
 'process',
 '(',
 'int',
 'option',
 ',',
 'int',
 'a',
 ',',
 'int',
 'b',
 ')',
 '{',
 'int',
 '*',
 'x',
 ';',
 'int',
 'k',
 '=',
 'a',
 '+',
 'b',
 ';',
 'int',
 'result',
 '=',
 'k',
 ';',
 'if',
 '(',
 'b',
 '!=',
 '0',
 ')',
 '{',
 'result',
 '=',
 'a',
 '/',
 'b',
 ';',
 '}',
 'else',
 '{',
 'printf',
 '(',
 '"Error: Division by zero is undefined.\n"',
 ')',
 ';',
 'return',
 '-1',
 ';',
 '// Return an error code',
 '}',
 'switch',
 '(',
 'option',
 ')',
 '{',
 'case',
 '1',
 ':',
 'result',
 '=',
 'a',
 '+',
 'b',
 '+',
 'c',
 ';',
 'break',
 ';',
 'case',
 '2',
 ':',
 'result',
 '=',
 'a',
 '-',
 'c',
 ';',
 'break',
 ';',
 'case',
 '3',
 ':',
 'result',
 '=',
 'a',
 '*',
 'c',
 ';',
 'break',
 ';',
 'default',
 ':',
 'printf',
 '(',
 '"Error: Invalid option. Please choose a valid option (1-4).\n"',
 ')',
 ';',
 'return',
 '-1',
 ';',
 '// Return an error code',
 '}',
 'return',
 'result',
 ';',
 '}'